In [1]:
import json
import pandas as pd
import os
import gzip
import re

In [2]:
def read_data(files, path, keyword):
    data = []
    for file in files:
        if keyword in file:
            with gzip.open(path + file) as f:
                for line in f.readlines():
                    if 'Received prepared message'.encode('utf-8') in line:
                        data.append(json.loads(line))
    return data

In [3]:
def data_processing(data):
    df = pd.DataFrame(data)
    df['time'] = pd.to_datetime(df['time'], format = '%Y-%m-%dT%H:%M:%S.%f')
    df.sort_values(by=['time','message'], inplace = True)
    df.reset_index(inplace = True, drop = True)
    df['viewID-diff'] = df['MsgViewID']-df['MsgViewID'].shift()
    return df

In [8]:
def getViewIdDiff(nodes):
    node_dict = {}
    node_lst = list(nodes)
    for i in range(len(nodes)):
        try:
            data = read_data(files, log_dir, node_lst[i])
        except:
            print("can't read data", node_lst[i])
        try:
            df = data_processing(data)
        except:
            print("can't process data", node_lst[i])
        count = df.groupby('viewID-diff')['viewID-diff'].count()
        node_dict[node_lst[i]] = count
    return pd.DataFrame(node_dict)

In [5]:
log_dir = "/home/ubuntu/jupyter/logs/os/20/04/14/19:35:10/validator/tmp_log/log-20200405.194121/"
files = os.listdir(log_dir)
nodes = set()
pattern = re.compile("zerolog-validator-(.*?)-9000")
for file in files:
    nodes.add(re.findall(pattern, file)[0])

In [9]:
getViewIdDiff(nodes)

,13.57.48.226,13.56.255.41,13.56.228.189,13.57.198.215,107.20.95.47,13.56.179.90,13.56.240.112,13.57.9.233,13.56.194.232
viewID-diff,,,,,,,,,
1.0,8800,8777,6123,9060,6122,6109,9063,6152,6060
